In [1]:
import pandas as pd
df = pd.read_csv("CycPeptMPDB_Peptide_Assay_PAMPA.csv", low_memory=False)

In [2]:
df = df.iloc[:, 18:]

In [3]:
columns_to_drop = df.columns[1:16]
df = df.drop(columns=columns_to_drop)

In [4]:
def convert_columns_to_float(dfn):
    for col in dfn.columns:
        try:
            dfn[col] = dfn[col].astype(float)
        except ValueError:
            dfn.drop(columns=[col], inplace=True)
    return dfn
    
df = convert_columns_to_float(df)

In [5]:
df

,Permeability,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,...,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea,PC1,PC2
0,-6.60,15.129540,-1.609940,15.129540,0.022871,0.147925,1202.635,1090.747,1201.841368,486.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.071536,-0.870594
1,-6.20,14.157239,-1.126245,14.157239,0.000719,0.215270,712.933,652.453,712.452348,284.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.129957,-1.415233
2,-7.00,14.157239,-1.126245,14.157239,0.000719,0.215270,712.933,652.453,712.452348,284.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.129957,-1.415233
3,-7.10,14.157239,-1.126245,14.157239,0.000719,0.215270,712.933,652.453,712.452348,284.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.129957,-1.415233
4,-7.20,14.157239,-1.126245,14.157239,0.000719,0.215270,712.933,652.453,712.452348,284.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.129957,-1.415233
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6936,-10.00,14.769384,-1.347468,14.769384,0.007866,0.059792,918.113,854.609,917.491193,356.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-2.574465,12.417057
6937,-6.82,14.464936,-1.224147,14.464936,0.024093,0.088837,824.022,766.566,823.416314,316.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-3.177717,12.512638
6938,-6.40,14.668139,-1.249432,14.668139,0.015079,0.073539,900.120,838.632,899.447614,344.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-2.763162,13.866094
6939,-6.59,14.775255,-1.262058,14.775255,0.010403,0.059304,950.180,886.676,949.463264,362.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-2.589502,15.443176


In [6]:
df = df[df.Permeability != -10]

In [7]:
def classify_permeability(value):
    if value >= -5:
        return 'good'
    elif -7 < value < -5:
        return 'moderate/low'
    else:
        return 'impermeable'

df['Permeability'] = df['Permeability'].apply(classify_permeability)

# Display the distribution of the new classes
print(df['Permeability'].value_counts())

Permeability
moderate/low    5260
good            1033
impermeable      408
Name: count, dtype: int64


C:\Users\hj728\AppData\Local\Temp\ipykernel_18520\4119973859.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Permeability'] = df['Permeability'].apply(classify_permeability)


In [8]:
from sklearn.model_selection import train_test_split

# Assuming `data` is your DataFrame and `target` is the target column
X = df.drop(columns=['Permeability'])
y = df['Permeability']

X.drop(['PC1', 'PC2'], axis = 1, inplace = True)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

mapping = dict(zip(le.classes_, range(len(le.classes_))))
print("Mapping of categories to encoded values:", mapping)

# Split the data into 80% train, 20% temporary (to be split into test and validation)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

Mapping of categories to encoded values: {'good': 0, 'impermeable': 1, 'moderate/low': 2}


In [9]:
from sklearn.model_selection import cross_val_score, cross_val_predict, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.metrics import accuracy_score, f1_score, classification_report
import numpy as np

In [10]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [15]:
import numpy as np
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import accuracy_score, f1_score, classification_report, make_scorer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Custom Keras wrapper
class KerasClassifierWrapper:
    def __init__(self, build_fn, epochs=50, batch_size=10, verbose=0):
        self.build_fn = build_fn
        self.epochs = epochs
        self.batch_size = batch_size
        self.verbose = verbose
        self.model = None

    def fit(self, X, y):
        self.model = self.build_fn()
        self.model.fit(X, y, epochs=self.epochs, batch_size=self.batch_size, verbose=self.verbose)
        return self

    def predict(self, X):
        return np.argmax(self.model.predict(X), axis=-1)

    def get_params(self, deep=True):
        return {
            'build_fn': self.build_fn,
            'epochs': self.epochs,
            'batch_size': self.batch_size,
            'verbose': self.verbose
        }

    def set_params(self, **params):
        for param, value in params.items():
            setattr(self, param, value)
        return self

# Define the k-fold cross-validation
kfold = KFold(n_splits=3, shuffle=True, random_state=42)

# Custom scorer for F1 score
f1_scorer = make_scorer(f1_score, average='macro')

# Define models
models = {
    'RandomForest': RandomForestClassifier(),
    'LogisticRegression': LogisticRegression(max_iter=500),
    'XGBoost': XGBClassifier(),
    'Decision Tree': DecisionTreeClassifier(),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Support Vector Machine': SVC(probability=True),
}

# Function to create Keras model
def create_keras_model():
    model = Sequential()
    model.add(Dense(12, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(len(np.unique(y_train)), activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Add Keras model to models dictionary using the custom wrapper
models['Keras'] = KerasClassifierWrapper(build_fn=create_keras_model, epochs=50, batch_size=10, verbose=0)

# Perform cross-validation and evaluate each model
for name, model in models.items():
    # Cross-validation on the training set
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=f1_scorer)
    mean_f1_score = cv_results.mean()
    print(f'{name} - Mean F1 Score on Training Data: {mean_f1_score:.2f}')
    
    # Train the model on the entire training set
    model.fit(X_train, y_train)
    
    # Predict on the test set
    y_pred = model.predict(X_test)
    
    # Evaluate the model on the test set
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='macro')
    report = classification_report(y_test, y_pred)
    
    print(f'{name} - Test Accuracy: {accuracy:.2f}')
    print(f'{name} - Test F1 Score: {f1:.2f}')
    print(f'{name} - Classification Report:\n{report}')
    print('-' * 50)

RandomForest - Mean F1 Score on Training Data: 0.75
RandomForest - Test Accuracy: 0.90
RandomForest - Test F1 Score: 0.77
RandomForest - Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.72      0.79       109
           1       0.79      0.45      0.58        42
           2       0.90      0.97      0.93       520

    accuracy                           0.90       671
   macro avg       0.85      0.71      0.77       671
weighted avg       0.89      0.90      0.89       671

--------------------------------------------------
LogisticRegression - Mean F1 Score on Training Data: 0.68
LogisticRegression - Test Accuracy: 0.87
LogisticRegression - Test F1 Score: 0.70
LogisticRegression - Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.63      0.73       109
           1       0.76      0.31      0.44        42
           2       0.88      0.97      0.92       520

    ac

d:\Python\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


d:\Python\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  


d:\Python\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Keras - Mean F1 Score on Training Data: 0.74


d:\Python\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Keras - Test Accuracy: 0.89
Keras - Test F1 Score: 0.77
Keras - Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.72      0.77       109
           1       0.80      0.48      0.60        42
           2       0.91      0.96      0.93       520

    accuracy                           0.89       671
   macro avg       0.84      0.72      0.77       671
weighted avg       0.89      0.89      0.88       671

--------------------------------------------------
